In [6]:
string = '1' + '2' + '3'
string_strip = string.strip(' ')
print('string_strip: ', string_strip)
string_strip_split = string_strip.split(' ')
print('string_strip_split: ', string_strip_split)

string_strip:  123
string_strip_split:  ['123']


In [18]:
!kaggle datasets list -s UNIFESP 

ref                                                         title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
felipekitamura/unifesp-xray-bodypart-classification         The UNIFESP X-Ray Body Part Classification Dataset  256MB  2022-04-03 01:56:15            654         30  1.0              
ibombonato/xray-body-images-in-png-unifesp-competion        Body Parts X-Ray Images in PNG                      200MB  2022-04-11 17:03:47            764         23  0.7941176        
eferrares/unifesp-contest-data                              Unifesp AM Classes                                   16KB  2017-04-27 02:07:39             49          1  0.64705884       
ibombonato/unifesp-xray-body-part-dicom-metadata-csv        UNIFESP X-ray Body P

In [19]:
!kaggle datasets download ibombonato/xray-body-image-in-png-unifesp-competion

403 - Forbidden - Permission 'datasets.get' was denied
